# SPIDER: Scanning Probe microscopy Image DEnoising and Reconstruction.

Scanning Probe microscopy Image DEnoising and Reconstruction (SPIDER) is a self-supervised image denoising and reconstruction algorithm dedicated for scanning probe micropy (SPM). This notebook gives an intutive and convenient way to perform SPIDER on demo or custom datasets. The SPIDER consists of three modules:

1. hysteresis correction
2. self-supervised denoising
3. (optional) eqvariant reconstruction

If you use SPIDER, please cite the following paper:

- Sichen Pan, Simon Scheuring. "[Self-supervised denoising and restoration method for atomic force microscopy]()" In preparation (2026)

In [ ]:
# run this cell to enable form fields in local Jupyter Notebooks
%load_ext ipyform
%form_config --auto-detect 1
%matplotlib inline

In [ ]:
import subprocess
def run_command(cmd, verbose=False):
  """
  :param cmd: list or str, command
  :param verbose: bool, whether to print output
  """
  print(f"Running: {cmd}")
  process = subprocess.Popen(cmd, shell=isinstance(cmd, str),
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    text=True, bufsize=1)
  output_lines = []
  for line in process.stdout:
    if verbose:
      print(line, end="")
    output_lines.append(line)
  process.wait()
  print(f"Return code: {process.returncode}")
  output = "".join(output_lines)
  if process.returncode != 0:
    raise subprocess.CalledProcessError(process.returncode, cmd, output=output)

## Hysteresis correction
This section performs subpixel alignment of AFM columns between trace and retrace images to correct piezo hysteresis effect.

### 1. file path
- `raw_path` should contain both raw trace and retrace images. The file name of `trace`(`retrace`) should contain `trace` or `1Ch` (`retrace` or `1ChRet`).
- corrected images will be saved in `datasets`  

In [ ]:
raw_path = "./raw" # @param {type:"string", placeholder:"raw images path"}
data_path = "./datasets" # @param {type:"string", placeholder:"dataset path for training"}

### 2. adjustable parameters for hysteresis correction
- `shift`: the shiftment of the attention window
- `window`: the window size
- `trace_left`(`trace_right`): the boundary ratio used in calculation
- `save`: if true, quadric fitted parameters and values results will be saved in `data_path`

In [ ]:
shift = 0 # @param {type:"slider", min:-0.5, max:0.5, step:0.01}
window = 0.1 # @param {type:"slider", min:0.01, max:0.5, step:0.01}
trace_left = 0.05 # @param {type:"slider", min:0.0, max:0.5, step:0.01}
trace_right = 0.95 # @param {type:"slider", min:0.5, max:1, step:0.01}
save = False # @param {type:"boolean"}

# @markdown usually it is not necessary to adjust below parameters
retrace_left = 0. # @param {type:"slider", min:0.0, max:1, step:0.01}
retrace_right = 1. # @param {type:"slider", min:0.0, max:1, step:0.01}
temperature = 0.01 # @param {type:"number"}

from preprocess import data_generator
data_gen = data_generator(raw_path, data_path, shift=shift, window=window, tl=trace_left, tr=trace_right, rl=retrace_left, rr=retrace_right, tem=temperature, save=save)
data_gen.generate()

## Self-supervised denoising
This section performs self-supervised denoising step for SPM images. The algorithm is implemented with Distributed Data Parallel (DDP), which allows users to run it with multiple GPUs. 

### 1. file path
- `checkpoint_path`: path to save checkpoints
- `prediction_path`: path to save predicited images

In [ ]:
checkpoint_path = "./checkpoints" # @param {type:"string", placeholder:"checkpoint path"}
prediction_path = "./predictions" # @param {type:"string", placeholder:"prediction path"}

### 2. hyperparameters for denoising
adjustable basic parameters for self-supervised denoising. If scale-up runs or custom modification are needed, running through command lines with more adjustable parameters is recommended.
- `batch_size`
- `epochs`
- `patch_size`: pixel size of which an image is divided into
- `augmentation`
- `ensembles`: ensembels runs
- `world_size`: GPU numbers


In [ ]:
batch_size = 512 # @param {type:"integer"}
epochs = 50 # @param {type:"integer"}
patch_size = 64 # @param {type:"integer"}
augmentation = True # @param {type:"boolean"}
ensembles = 3 # @param {type:"integer"}
world_size = 1 # @param {type:"integer"}


In [ ]:
run_command(f"python trainer_ddp.py -dp {data_path} -cp {checkpoint_path} -b {batch_size} -e {epochs} -ps {patch_size} {'-a' if augmentation else ''} -n {ensembles} -ws {world_size}", verbose=True)

In [ ]:
import plotly.express as px
from skimage import io
import numpy as np

img = io.imread('/Users/psichen/Library/CloudStorage/OneDrive-med.cornell.edu/deepAFM/WS/dualAFM/predictions/output_last_pix.tif')
trace = io.imread('/Users/psichen/Library/CloudStorage/OneDrive-med.cornell.edu/deepAFM/WS/dualAFM/predictions/trace_pix.tif')
retrace = io.imread('/Users/psichen/Library/CloudStorage/OneDrive-med.cornell.edu/deepAFM/WS/dualAFM/predictions/retrace_pix.tif')
imgs = np.concatenate([trace[np.newaxis,...], img[np.newaxis,...], retrace[np.newaxis,...]], axis=0)
fig = px.imshow(imgs, facet_col=0, animation_frame=1, labels=dict(animation_frame="frame"), color_continuous_scale='gray', zmin=np.min(img), zmax=np.max(img))
fig.show()